In [ ]:
!pip install -U agno google-genai

In [ ]:
import os

# Google GenAI API Key 설정
os.environ["GOOGLE_API_KEY"] = "{{KEY}}"
os.environ["OPENAI_API_KEY"] = "{{KEY}}"
print("Environment variables set.")

In [ ]:
# 1. Google Drive 마운트 (최초 1회 실행 필요)
from google.colab import drive
import os
from agno.team import Team
from agno.agent import Agent
from agno.models.google import Gemini
from agno.media import Image

drive.mount('/content/drive')

IMAGE_PATH = "/content/drive/MyDrive/eiffel_tower.jpg"

# 이미지가 실제로 있는지 확인
if not os.path.exists(IMAGE_PATH):
    print(f"⚠️ 경고: '{IMAGE_PATH}' 경로에 파일이 없습니다. 경로를 확인해주세요.")
else:
    print(f"✅ 이미지 확인 완료: {IMAGE_PATH}")

model_id = "gemini-flash-lite-latest"

# 1. 각국 전문가 (텍스트 기반 처리)
france = Agent(
    name="France Professional",
    role="France Professional (Return in French)",
    model=Gemini(id=model_id),
    instructions=["프랑스 관련 질문이나 묘사가 들어오면 무조건 **불어**로 대답해."]
)

usa = Agent(
    name="USA Professional",
    role="USA Professional (Return in English)",
    model=Gemini(id=model_id),
    instructions=["미국 관련 질문이나 묘사가 들어오면 무조건 **영어**로 대답해."]
)

korea = Agent(
    name="South Korea Professional",
    role="South Korea Professional (Return in Korean)",
    model=Gemini(id=model_id),
    instructions=["한국 관련 질문이나 묘사가 들어오면 무조건 **한국어**로 대답해."]
)

# 2. 멀티모달 팀 (이미지를 보고 라우팅하는 역할)
team = Team(
    name="Visual Analysis Team",
    model=Gemini(id=model_id),
    members=[france, usa, korea],
    instructions=[
        "당신은 이미지를 분석하고 적절한 국가 전문가에게 연결하는 팀 리더입니다.",
        "1. 제공된 이미지를 시각적으로 분석하여 어느 나라와 관련이 있는지 판단하세요.",
        "2. 해당 국가의 전문가 에이전트에게 **이미지에 대한 상세한 묘사**를 전달하고 설명을 요청하세요.",
        "3. 사용자가 별말 없이 이미지만 줘도, 알아서 랜드마크나 특징을 파악해야 합니다.",
        "4. 직접 대답하지 말고 반드시 멤버 에이전트를 호출하세요."
    ],
    markdown=True
)

# 3. 멀티모달 검증 에이전트 (이미지 + 답변 대조)
review_agent = Agent(
    name="Visual & Language Auditor",
    model=Gemini(id=model_id),
    instructions=[
        "당신은 AI 팀의 답변을 검증하는 멀티모달 QA 담당자입니다.",
        "**입력으로 주어지는 '이미지'와 팀의 '텍스트 답변'을 비교 분석하세요.**",
        "다음 기준을 엄격히 평가하세요:",
        "1. **시각적 정확성 (Visual Grounding):** 답변이 실제 이미지 속 대상을 올바르게 설명하고 있는가? (예: 에펠탑 사진인데 자유의 여신상 설명 금지)",
        "2. **국가별 언어 규칙:**",
        "   - 프랑스에 대한 질문 질문 -> 불어 답변인가?",
        "   - 미국에 대한 질문 -> 영어 답변인가?",
        "   - 대한민국에 대한 질문 -> 한국어 답변인가?",
        "3. **결과 리포트:** 'PASS' 또는 'FAIL'과 그 이유를 한국어로 명확히 작성하세요."
    ],
    markdown=True
)

# --- 실행 로직 ---
user_prompt = "이 사진에 대해서 설명해줘."

team_response = team.run(
    user_prompt,
    images=[Image(filepath=IMAGE_PATH)]
)
generated_content = team_response.content

print(f"📝 [Step 1 결과] Team 답변:\n{generated_content}\n")